### Code Generation using Amazon Bedrock 


Given below is a example for code generation use case. In this notebook, we will leverage Anthropic Claude-3 Sonnet Model available on Amazon Bedrock to adopt the role of a python developer and perform the code-generation task.

In [14]:
# Import the respective libraries
import boto3
import botocore
import os
import json
import sys

#Create client side Amazon Bedrock connection with Boto3 library
region = os.environ.get("AWS_REGION")
bedrock_runtime = boto3.client(service_name='bedrock-runtime',region_name=region)

# Adding prompt example here:
prompt_data = """Human: You are an expert Python developer tasked with coding a web scraper for an experienced developer. The scraper should extract data from multiple web pages and store the results in a SQLite database. Write clean, high-quality Python code for this task, including necessary imports. Do not write anything before the ```python block. After writing the code, carefully check for errors. If errors exist, list them within <error> tags and provide a new corrected version. If no errors, write "CHECKED: NO ERRORS" within <error> tags.
Assistant:
"""

# Using Messages API with Anthropic Claude
messages=[{ "role":'user', "content":[{'type':'text','text': prompt_data}]}]

body=json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31", 
            "max_tokens": 512,
            "messages": messages,
            "temperature": 0.1,
            "top_p": 1
        }  
    )  
    
response = bedrock_runtime.invoke_model(body=body, modelId="anthropic.claude-3-sonnet-20240229-v1:0")
response_body = json.loads(response.get('body').read())


In [18]:
print(response_body['content'][0].get('text'))

```python
import requests
from bs4 import BeautifulSoup
import sqlite3

# Connect to SQLite database
conn = sqlite3.connect('scraped_data.db')
c = conn.cursor()

# Create table if it doesn't exist
c.execute('''CREATE TABLE IF NOT EXISTS scraped_data
             (url TEXT, title TEXT, content TEXT)''')

# List of URLs to scrape
urls = ['https://www.example.com/page1', 'https://www.example.com/page2', 'https://www.example.com/page3']

# Loop through URLs and scrape data
for url in urls:
    # Send HTTP request and get response
    response = requests.get(url)
    
    # Parse HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extract title and content
    title = soup.find('h1').text
    content = soup.find('div', {'class': 'content'}).text
    
    # Insert data into SQLite table
    c.execute("INSERT INTO scraped_data VALUES (?, ?, ?)", (url, title, content))

# Commit changes and close connection
conn.commit()
conn.close()
```

<erro